In [1]:
import pandas as pd
import funciones.test_func as tf
from sqlalchemy import create_engine
import os
from dotenv import load_dotenv

load_dotenv()

host=os.getenv("DB_HOST2")
port=os.getenv("DB_PORT2")
database=os.getenv("DB_NAME2")
user=os.getenv("DB_USER2")
password=os.getenv("DB_PASS2")

engine = create_engine(f'postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}')

tables = tf.excecute_query_tables(engine)

dfs = tf.create_df_variable(tables, engine)
patients = dfs['patients']
doctors= dfs['doctors']
appointments = dfs['appointments']
treatments = dfs['treatments']

In [2]:
patients

,patient_id,name,gender,birth_date
0,1,Ana Torres,F,1985-07-10
1,2,Luis Martínez,M,1970-02-20
2,3,Carlos Gómez,M,1995-11-15
3,4,Marta Ruiz,F,2001-01-05
4,5,Julia Herrera,F,1968-04-28
5,6,Pedro López,M,1988-06-12
6,7,Sandra Díaz,F,1992-09-30
7,8,Andrés Pérez,M,1975-12-19
8,9,Laura Castro,F,2000-03-21
9,10,José Molina,M,1963-08-08


In [3]:
doctors

,doctor_id,name,specialty
0,1,Dr. Pérez,Cardiology
1,2,Dr. Rojas,Neurology
2,3,Dr. Torres,General
3,4,Dr. Álvarez,Oncology
4,5,Dr. Salinas,Dermatology
5,6,Dr. Fernández,Pediatrics


In [4]:
appointments

,appointment_id,patient_id,doctor_id,appointment_date,status
0,101,1,1,2023-05-10,Completed
1,102,2,2,2023-05-15,Completed
2,103,3,3,2023-06-01,Cancelled
3,104,1,1,2023-06-20,Completed
4,105,4,2,2023-06-22,Completed
5,106,5,4,2023-07-05,Completed
6,107,5,4,2023-07-20,Completed
7,108,2,1,2023-08-10,Completed
8,109,6,5,2023-09-01,Cancelled
9,110,7,3,2023-09-10,Completed


In [5]:
treatments

,treatment_id,patient_id,illness,start_date,end_date,outcome
0,201,1,Hypertension,2023-05-10,2023-07-01,Recovered
1,202,2,Hypertension,2023-05-15,2023-08-15,Recovered
2,203,5,Cancer,2023-07-05,2023-10-10,Deceased
3,204,4,Migraine,2023-06-22,2023-07-22,Recovered
4,205,6,Cancer,2023-09-01,2023-10-01,Deceased
5,206,7,Diabetes,2023-09-10,2023-11-15,Ongoing
6,207,8,Leukemia,2023-09-12,2023-12-20,Ongoing
7,208,9,Cancer,2023-10-01,2023-11-01,Deceased
8,209,10,Diabetes,2023-10-15,2023-11-15,Recovered
9,210,11,Asthma,2023-10-20,2023-12-01,Recovered


🏥 Ejercicio 5 – Optimización de Recursos y Análisis Operativo en el Hospital
Contexto
Como analista de datos en el departamento de planeación hospitalaria, has sido asignado a identificar cuellos de botella y oportunidades de mejora en la gestión operativa de citas, distribución de pacientes y uso de recursos médicos. Para esto, trabajarás con información de pacientes, médicos, citas agendadas y tratamientos realizados. El objetivo de este análisis es proponer recomendaciones que optimicen los flujos de atención y mejoren la asignación de personal médico.

Los datos se encuentran distribuidos en las siguientes tablas:

- patients: contiene los datos generales de cada paciente.
- doctors: almacena la información de cada médico y su especialidad.
- appointments: incluye cada cita programada, con su estado y fecha.
- treatments: registra los tratamientos realizados, el doctor que los ejecutó, el paciente tratado, la enfermedad y su resultado.

📊 Preguntas del Challenge #5

1. Carga Operativa por Médico

a. ¿Cuántos tratamientos ha atendido cada médico?

In [6]:
#SQL

query = """
WITH treatments_ids AS (
SELECT treatment_id, patient_id	 
FROM treatments 
)

SELECT a.doctor_id, d.name, COUNT(DISTINCT a.patient_id) AS treatment_count
FROM appointments a
JOIN treatments_ids ti ON ti.patient_id = a.patient_id
JOIN doctors d ON d.doctor_id = a.doctor_id
GROUP BY a.doctor_id, d.name
ORDER BY COUNT(DISTINCT a.patient_id) DESC
"""

tf.excecute_query(query, engine)

,doctor_id,name,treatment_count
0,2,Dr. Rojas,5
1,1,Dr. Pérez,4
2,4,Dr. Álvarez,3
3,5,Dr. Salinas,2
4,6,Dr. Fernández,2
5,3,Dr. Torres,1


In [7]:
#Python

treatments_ids = pd.merge(pd.merge(treatments['patient_id'], appointments[['doctor_id', 'patient_id']], on='patient_id'), doctors[['doctor_id', 'name']], on='doctor_id')
treatments_ids = treatments_ids.groupby(['doctor_id', 'name'])['patient_id'].nunique().reset_index().sort_values(by='patient_id', ascending=False)
treatment_count_doctor = treatments_ids.rename(columns={'patient_id':'treatment_count'})
treatment_count_doctor

,doctor_id,name,treatment_count
1,2,Dr. Rojas,5
0,1,Dr. Pérez,4
3,4,Dr. Álvarez,3
4,5,Dr. Salinas,2
5,6,Dr. Fernández,2
2,3,Dr. Torres,1


b. ¿Cuántas citas médicas tiene asignadas cada médico en total?

In [8]:
#SQL

query = """
WITH appointments_doctors AS (
SELECT doctor_id, COUNT(appointment_id) AS total_appointments
FROM appointments
GROUP BY doctor_id
)

SELECT d.doctor_id, d.name, ap.total_appointments
FROM doctors d
JOIN appointments_doctors ap ON ap.doctor_id = d.doctor_id
ORDER BY total_appointments DESC
"""

tf.excecute_query(query, engine)

,doctor_id,name,total_appointments
0,4,Dr. Álvarez,9
1,1,Dr. Pérez,8
2,2,Dr. Rojas,7
3,3,Dr. Torres,6
4,5,Dr. Salinas,4
5,6,Dr. Fernández,2


In [9]:
#Python

appointments_doctors = appointments.groupby('doctor_id')['appointment_id'].count().reset_index()
appointments_doctors = appointments_doctors.rename(columns={'appointment_id':'total_appointments'})
appointments_doctors = pd.merge(doctors[['doctor_id','name']], appointments_doctors, on='doctor_id')
appointments_count_doctor = appointments_doctors.sort_values(by='total_appointments', ascending=False)
appointments_count_doctor

,doctor_id,name,total_appointments
3,4,Dr. Álvarez,9
0,1,Dr. Pérez,8
1,2,Dr. Rojas,7
2,3,Dr. Torres,6
4,5,Dr. Salinas,4
5,6,Dr. Fernández,2


c. ¿Qué médico tiene la mayor carga operativa (suma de tratamientos + citas)?

In [10]:
#SQL

query = """
WITH treatments_ids AS (
SELECT treatment_id, patient_id	 
FROM treatments 
),

treatment_count_doctor AS (
SELECT a.doctor_id, d.name, COUNT(DISTINCT a.patient_id) AS treatment_count
FROM appointments a
JOIN treatments_ids ti ON ti.patient_id = a.patient_id
JOIN doctors d ON d.doctor_id = a.doctor_id
GROUP BY a.doctor_id, d.name
ORDER BY COUNT(DISTINCT a.patient_id) DESC
),

appointments_doctors AS (
SELECT doctor_id, COUNT(appointment_id) AS total_appointments
FROM appointments
GROUP BY doctor_id
),

appointments_count_doctor AS (
SELECT d.doctor_id, d.name, ap.total_appointments
FROM doctors d
JOIN appointments_doctors ap ON ap.doctor_id = d.doctor_id
ORDER BY total_appointments DESC
)


SELECT tcd.name, acd.total_appointments, tcd.treatment_count, (acd.total_appointments + tcd.treatment_count) AS operational_load
FROM treatment_count_doctor tcd
JOIN appointments_count_doctor acd ON tcd.doctor_id = acd.doctor_id
ORDER BY operational_load DESC
"""

tf.excecute_query(query, engine)

,name,total_appointments,treatment_count,operational_load
0,Dr. Pérez,8,4,12
1,Dr. Rojas,7,5,12
2,Dr. Álvarez,9,3,12
3,Dr. Torres,6,1,7
4,Dr. Salinas,4,2,6
5,Dr. Fernández,2,2,4


In [11]:
#Python
operational_load = pd.merge(treatment_count_doctor, appointments_count_doctor[['doctor_id','total_appointments']], on = 'doctor_id')
operational_load['operational_load'] = operational_load['total_appointments'] + operational_load['treatment_count']
operational_load = operational_load[['name', 'treatment_count',	'total_appointments', 'operational_load']].sort_values(by='operational_load', ascending=False)
operational_load

,name,treatment_count,total_appointments,operational_load
0,Dr. Rojas,5,7,12
1,Dr. Pérez,4,8,12
2,Dr. Álvarez,3,9,12
5,Dr. Torres,1,6,7
3,Dr. Salinas,2,4,6
4,Dr. Fernández,2,2,4


2. Análisis de Frecuencia de Enfermedades

a. ¿Cuáles son las enfermedades más comunes tratadas en el hospital?

In [12]:
#SQL

query = """
SELECT illness, COUNT(treatment_id) as count_illness
FROM treatments
GROUP BY illness
ORDER BY count_illness DESC
LIMIT 1
"""

tf.excecute_query(query, engine)

,illness,count_illness
0,Cancer,4


In [13]:
#Python
df_illnedd_count = treatments.groupby('illness')['treatment_id'].count().reset_index()
df_illnedd_count = df_illnedd_count.rename(columns={'treatment_id':'count_illness'}).sort_values(by='count_illness', ascending=False)
df_illnedd_count.head(1)


,illness,count_illness
1,Cancer,4


b. ¿Qué enfermedad tiene el mayor número de pacientes fallecidos?

In [14]:
#SQL

query = """
SELECT illness, COUNT(treatment_id) as count_deceased
FROM treatments
WHERE outcome = 'Deceased'
GROUP BY illness
ORDER BY count_deceased DESC
LIMIT 1
"""

tf.excecute_query(query, engine)

,illness,count_deceased
0,Cancer,4


In [15]:
#Python

df_deceased = treatments[treatments['outcome'] == 'Deceased']
df_deceased = df_deceased.groupby('illness')['treatment_id'].count().reset_index()
df_deceased = df_deceased.rename(columns={'treatment_id':'count_deceased'}).sort_values(by='count_deceased', ascending=False)
df_deceased.head(1)

,illness,count_deceased
0,Cancer,4


c. ¿Qué enfermedad presenta el mayor porcentaje de recuperación?

In [16]:
#SQL

query = """
WITH recover_count AS (
SELECT illness, COUNT(treatment_id) as count_recovered
FROM treatments
WHERE outcome = 'Recovered'
GROUP BY illness
ORDER BY count_recovered DESC
),

illness_count AS (
SELECT illness, COUNT(treatment_id) as count_illness
FROM treatments
GROUP BY illness
ORDER BY count_illness DESC
),

final_dataframe AS (
SELECT ic.illness, count_illness, count_recovered, (count_recovered * 1.0 / count_illness) AS recover_percentaje
FROM illness_count ic 
JOIN recover_count rc ON ic.illness = rc.illness 
ORDER BY recover_percentaje DESC
)

--Acá hay un empate, se deben mostrar todos los que estén en el primer lugar
SELECT illness, count_illness, count_recovered, recover_percentaje
FROM final_dataframe
WHERE recover_percentaje = (SELECT MAX(recover_percentaje) FROM final_dataframe)
"""

tf.excecute_query(query, engine)

,illness,count_illness,count_recovered,recover_percentaje
0,Asthma,1,1,1.0
1,Migraine,1,1,1.0


In [17]:
#Python

df_recovered = treatments[treatments['outcome'] == 'Recovered']
df_recovered = df_recovered.groupby('illness')['treatment_id'].count().reset_index()
df_recovered = df_recovered.rename(columns={'treatment_id':'count_recovered'}).sort_values(by='count_recovered', ascending=False)

df_illness = treatments.groupby('illness')['treatment_id'].count().reset_index()
df_illness = df_illness.rename(columns={'treatment_id':'count_illness'}).sort_values(by='count_illness', ascending=False)

df = pd.merge(df_recovered, df_illness, on='illness')
df['recover_percentaje'] = df['count_recovered'] / df['count_illness']
df = df.sort_values(by='recover_percentaje', ascending=False)

# Acá hay un empate, se deben mostrar todos los que estén en el primer lugar
df[df['recover_percentaje'] == df['recover_percentaje'].max()]

,illness,count_recovered,count_illness,recover_percentaje
1,Asthma,1,1,1.0
3,Migraine,1,1,1.0


3. Desempeño por Edad y Género

a. ¿Cuál es la tasa de recuperación por grupo etario?

In [51]:
#SQL

query = """
WITH ages AS (
SELECT  name, patient_id, EXTRACT(YEAR FROM AGE(CURRENT_DATE, birth_date::date)) AS patient_age
FROM patients
),

age_group AS (
SELECT *,
CASE WHEN patient_age <= 18 THEN 'kid' 
WHEN patient_age > 18 AND patient_age < 30 THEN 'young_adult' 
WHEN patient_age >= 30 AND patient_age < 60 THEN 'old_adult' 
ELSE 'old_person' END AS age_group
FROM ages 
),

recovered AS (
SELECT a.age_group, COUNT(t.outcome) AS count_recovered
FROM treatments t
JOIN age_group a ON a.patient_id = t.patient_id
WHERE outcome = 'Recovered'
GROUP BY a.age_group
),

treated AS (
SELECT a.age_group, COUNT(t.outcome) AS count_treated
FROM treatments t
JOIN age_group a ON a.patient_id = t.patient_id
GROUP BY a.age_group
)

SELECT tr.age_group, count_treated, count_recovered, (count_recovered * 1.0 / count_treated) AS recovery_rate
FROM recovered r
JOIN  treated tr ON r.age_group = tr.age_group
ORDER BY recovery_rate DESC
"""

tf.excecute_query(query, engine)

,age_group,count_treated,count_recovered,recovery_rate
0,old_person,1,1,1.000000
1,old_adult,8,3,0.375000
2,young_adult,3,1,0.333333


In [ ]:
#Python

from datetime import date

ages = patients.copy()
ages['birth_date'] = pd.to_datetime(ages['birth_date'])
ages['patient_year_birth'] = ages['birth_date'].dt.year 
today = pd.to_datetime(date.today()).year
ages['patient_age'] = today - ages['patient_year_birth']

ages['age_group'] = 'old_person'
ages.loc[ages['patient_age'] <= 18, 'age_group'] = 'kid'
ages.loc[(ages['patient_age'] > 18) & (ages['patient_age'] < 30), 'age_group'] = 'young_adult'
ages.loc[(ages['patient_age'] >= 30) & (ages['patient_age'] < 60), 'age_group'] = 'old_adult'

treatments_general = pd.merge(treatments, ages, on='patient_id')  
treatments_total = treatments_general.groupby('age_group')['outcome'].count().reset_index()
treatments_total = treatments_total.rename(columns={'outcome':'count_treated'})

recovered = treatments_general[treatments_general['outcome'] == 'Recovered']
recovered = recovered.groupby('age_group')['outcome'].count().reset_index()
recovered = recovered.rename(columns={'outcome':'count_recovered'})

recovery_rate = pd.merge(treatments_total, recovered, on='age_group')
recovery_rate['recovery_rate'] = recovery_rate['count_recovered'] / recovery_rate['count_treated']
recovery_rate = recovery_rate.sort_values(by='recovery_rate', ascending=False)
recovery_rate


,age_group,count_treated,count_recovered,recovery_rate
1,old_person,1,1,1.000000
0,old_adult,8,3,0.375000
2,young_adult,3,1,0.333333


b. ¿Existen diferencias significativas en la tasa de recuperación entre hombres y mujeres?

In [ ]:
#SQL

query = """

"""

tf.excecute_query(query, engine)

In [ ]:
#Python



c. ¿Qué rango de edad tiene más pacientes en tratamiento activo actualmente?

In [ ]:
#SQL

query = """

"""

tf.excecute_query(query, engine)

In [ ]:
#Python

